In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF
import os
import math
import random
import pyarrow.parquet as pq
import copy
import optuna
from optuna.trial import TrialState
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)

Using device: cuda
Using device: cuda


In [3]:
train = pd.read_csv("/kaggle/input/asl-signs/train.csv")
import json
def read_json(path):
    with open(path, "r") as file:
        json_data = json.load(file)
    return json_data
s2p_map = read_json(os.path.join("/kaggle/input/asl-signs/sign_to_prediction_index_map.json"))
p2s_map = {v: k for k, v in s2p_map.items()}

encoder = lambda x: s2p_map.get(x)
decoder = lambda x: p2s_map.get(x)

train["label"] =train["sign"].map(encoder)
train = train.drop(["participant_id","sequence_id","sign"],axis=1)
train

,path,label
0,train_landmark_files/26734/1000035562.parquet,25
1,train_landmark_files/28656/1000106739.parquet,232
2,train_landmark_files/16069/100015657.parquet,48
3,train_landmark_files/25571/1000210073.parquet,23
4,train_landmark_files/62590/1000240708.parquet,164
...,...,...
94472,train_landmark_files/53618/999786174.parquet,238
94473,train_landmark_files/26734/999799849.parquet,108
94474,train_landmark_files/25571/999833418.parquet,86
94475,train_landmark_files/29302/999895257.parquet,188


In [4]:
def flattenData(path):
    df = pd.read_parquet(path)
    df = pad_or_truncate(df)
    flatten = df.to_numpy().flatten()
    return flatten

def pad_or_truncate(data):
    if len(data)>50:
        return truncate_start(data)
    elif len(data)<50:
        return pad_end(data)
    else:
        return data
def pad_end(data):
    rows_to_add = 50 - len(data)
    padded_data = {}
    for column in data.columns:
        padded_data[column] = [0] * rows_to_add
    return data.append(pd.DataFrame(padded_data))

def truncate_start(data):
    return data.iloc[:50, :]

In [5]:

y = train['label'].values
X = np.stack([flattenData("/kaggle/input/pca-data/pca_files/"+i) for i in tqdm(train['path'])])

100%|██████████| 94477/94477 [16:38<00:00, 94.60it/s] 


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [24]:
X_train = torch.FloatTensor(X_train).to(DEVICE)
X_test = torch.FloatTensor(X_test).to(DEVICE)
y_train = torch.tensor(y_train).to(DEVICE)
y_test = torch.tensor(y_test).to(DEVICE)
print("X shape: ",X.shape)
print("Y shape: ",y.shape)
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)

X shape:  (94477, 650)
Y shape:  (94477,)
X_train shape:  torch.Size([85029, 650])
y_train shape:  torch.Size([85029])
X_test shape:  torch.Size([9448, 650])
y_test shape:  torch.Size([9448])


In [25]:
X_train

tensor([[  8.8263,  -3.0656,  11.8574,  ...,   0.0000,   0.0000,   0.0000],
        [ 19.7643,   9.8388,  -4.0689,  ...,   0.0000,   0.0000,   0.0000],
        [-11.2891,   1.5575,  -6.3298,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [ 16.3869,   7.6579,   1.6181,  ...,   0.0000,   0.0000,   0.0000],
        [ 12.3552,  -4.7865,   4.8455,  ...,  -0.2096,  -0.4664,  -0.3443],
        [ 19.1033,   5.9618,  -0.5345,  ...,   0.0000,   0.0000,   0.0000]],
       device='cuda:0')

In [26]:
batch_size=256
train_loader = DataLoader(dataset = TensorDataset(X_train,y_train),batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset = TensorDataset(X_test,y_test) ,batch_size=batch_size, shuffle=False)

In [10]:
class FeedForward(nn.Module):
    def __init__(self,n_layers, nodes_list):
        super(FeedForward, self).__init__()
        self.n_layers = n_layers
        self.nodes_list = nodes_list
        
        
        
        
        self.layers = nn.ModuleList()
        
        
        for i in range(n_layers-2):
            self.layers.append(nn.Linear(self.nodes_list[i], self.nodes_list[i+1]))
            self.layers.append(nn.ReLU())
        
        self.layers.append(nn.Linear(self.nodes_list[-2],self.nodes_list[-1]))
        
        
        
    def forward(self, x):
        
        for layer in self.layers:
            x = layer(x)
        
        
        return x

In [11]:

num_classes = 250


one_layer_model64 = FeedForward(n_layers=3, nodes_list = [X_train.shape[1],64,num_classes]).to(device)
one_layer_model128 = FeedForward(n_layers=3, nodes_list = [X_train.shape[1],128,num_classes]).to(device)
two_layer_model64 = FeedForward(n_layers=4, nodes_list = [X_train.shape[1],64,64,num_classes]).to(device)
two_layer_model128 = FeedForward(n_layers=4, nodes_list = [X_train.shape[1],128,128,num_classes]).to(device)

custon = FeedForward(n_layers=8, nodes_list = [X_train.shape[1],128,128,256,256,128,128,num_classes]).to(device)

In [21]:
#method to train the model object passed
def train(model):
    
    epochs = 100
    lr = 0.001
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    criterion = nn.CrossEntropyLoss()
    
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    best_val_loss = float('inf')
    for epoch in range(epochs):
        train_loss = 0
        train_acc = 0
        for data, labels in train_loader:
            
            

            outputs = model(data)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            train_loss += loss.item()

            train_acc += torch.sum(preds == labels.data) 

        train_loss = train_loss / (len(train_loader)*batch_size)
        train_losses.append(train_loss)

        # Validation phase

        val_loss = 0.0
        val_acc = 0.0
        with torch.no_grad():
            for val_data, val_labels in val_loader:
                val_outputs = model(val_data)
                loss = criterion(val_outputs, val_labels)
                val_loss += loss.item()
                _, gt = torch.max(val_outputs, 1)
                val_acc += torch.sum(val_labels.data == gt)
            val_loss = val_loss / (len(val_loader)*batch_size)
            val_losses.append(val_loss)
            val_acc = val_acc/ (len(val_loader)*batch_size)

        # Update best model based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model.state_dict())

        # Print training progress
        if (epoch+1)%10==0:
            print('Epoch [%d/%d], Train Loss: %.4f, Val Loss: %.4f, Val acc: %.4f'
                  % (epoch+1, epochs, train_loss, val_loss, val_acc))
        
    return best_model, train_losses, val_losses



In [22]:
best_model1, train_losses1, val_losses1 = train(one_layer_model64)

  0%|          | 257/94477 [17:15<105:27:22,  4.03s/it]


Epoch [10/100], Train Loss: 0.0201, Val Loss: 0.0223, Val acc: 0.0073
Epoch [20/100], Train Loss: 0.0196, Val Loss: 0.0229, Val acc: 0.0078
Epoch [30/100], Train Loss: 0.0194, Val Loss: 0.0232, Val acc: 0.0072
Epoch [40/100], Train Loss: 0.0192, Val Loss: 0.0235, Val acc: 0.0088
Epoch [50/100], Train Loss: 0.0191, Val Loss: 0.0237, Val acc: 0.0082
Epoch [60/100], Train Loss: 0.0190, Val Loss: 0.0239, Val acc: 0.0088
Epoch [70/100], Train Loss: 0.0189, Val Loss: 0.0240, Val acc: 0.0073
Epoch [80/100], Train Loss: 0.0189, Val Loss: 0.0242, Val acc: 0.0075
Epoch [90/100], Train Loss: 0.0188, Val Loss: 0.0244, Val acc: 0.0077
Epoch [100/100], Train Loss: 0.0188, Val Loss: 0.0245, Val acc: 0.0087


In [25]:
best_model2, train_losses2, val_losses2 = train(two_layer_model64)


Epoch [10/100], Train Loss: 0.0200, Val Loss: 0.0219, Val acc: 0.0095
Epoch [20/100], Train Loss: 0.0194, Val Loss: 0.0227, Val acc: 0.0095
Epoch [30/100], Train Loss: 0.0190, Val Loss: 0.0233, Val acc: 0.0083
Epoch [40/100], Train Loss: 0.0188, Val Loss: 0.0238, Val acc: 0.0093
Epoch [50/100], Train Loss: 0.0186, Val Loss: 0.0242, Val acc: 0.0096
Epoch [60/100], Train Loss: 0.0185, Val Loss: 0.0246, Val acc: 0.0100
Epoch [70/100], Train Loss: 0.0184, Val Loss: 0.0251, Val acc: 0.0086
Epoch [80/100], Train Loss: 0.0183, Val Loss: 0.0252, Val acc: 0.0092
Epoch [90/100], Train Loss: 0.0182, Val Loss: 0.0257, Val acc: 0.0090
Epoch [100/100], Train Loss: 0.0181, Val Loss: 0.0260, Val acc: 0.0094


In [24]:
best_model3, train_losses3, val_losses3 = train(one_layer_model128)

Epoch [10/100], Train Loss: 0.0191, Val Loss: 0.0232, Val acc: 0.0071
Epoch [20/100], Train Loss: 0.0181, Val Loss: 0.0247, Val acc: 0.0071
Epoch [30/100], Train Loss: 0.0176, Val Loss: 0.0257, Val acc: 0.0067
Epoch [40/100], Train Loss: 0.0172, Val Loss: 0.0263, Val acc: 0.0070
Epoch [50/100], Train Loss: 0.0170, Val Loss: 0.0270, Val acc: 0.0064
Epoch [60/100], Train Loss: 0.0168, Val Loss: 0.0276, Val acc: 0.0062
Epoch [70/100], Train Loss: 0.0167, Val Loss: 0.0281, Val acc: 0.0064
Epoch [80/100], Train Loss: 0.0165, Val Loss: 0.0286, Val acc: 0.0069
Epoch [90/100], Train Loss: 0.0164, Val Loss: 0.0290, Val acc: 0.0069
Epoch [100/100], Train Loss: 0.0163, Val Loss: 0.0294, Val acc: 0.0063


In [23]:
best_model4, train_losses4, val_losses4 = train(two_layer_model128)

Epoch [10/100], Train Loss: 0.0189, Val Loss: 0.0228, Val acc: 0.0079
Epoch [20/100], Train Loss: 0.0173, Val Loss: 0.0250, Val acc: 0.0077
Epoch [30/100], Train Loss: 0.0164, Val Loss: 0.0268, Val acc: 0.0077
Epoch [40/100], Train Loss: 0.0159, Val Loss: 0.0284, Val acc: 0.0078
Epoch [50/100], Train Loss: 0.0155, Val Loss: 0.0299, Val acc: 0.0078
Epoch [60/100], Train Loss: 0.0152, Val Loss: 0.0313, Val acc: 0.0077
Epoch [70/100], Train Loss: 0.0149, Val Loss: 0.0328, Val acc: 0.0072
Epoch [80/100], Train Loss: 0.0147, Val Loss: 0.0339, Val acc: 0.0062
Epoch [90/100], Train Loss: 0.0145, Val Loss: 0.0354, Val acc: 0.0074
Epoch [100/100], Train Loss: 0.0144, Val Loss: 0.0369, Val acc: 0.0069


In [27]:
custombest, train_lossescustom, val_lossescustom = train(custon)

Epoch [10/100], Train Loss: 0.0200, Val Loss: 0.0217, Val acc: 0.0111
Epoch [20/100], Train Loss: 0.0182, Val Loss: 0.0247, Val acc: 0.0091
Epoch [30/100], Train Loss: 0.0163, Val Loss: 0.0291, Val acc: 0.0086
Epoch [40/100], Train Loss: 0.0145, Val Loss: 0.0338, Val acc: 0.0080
Epoch [50/100], Train Loss: 0.0132, Val Loss: 0.0390, Val acc: 0.0070
Epoch [60/100], Train Loss: 0.0122, Val Loss: 0.0447, Val acc: 0.0069


KeyboardInterrupt: 

In [31]:
learning_rate = 5e-4
weight_decay = 1e-1
cycle = 8

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(custon.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=cycle, eta_min=learning_rate / 10)

train_losses = []
val_losses = []

def train_val_loop(epoch, train_dataloader, val_dataloader, model, loss_fn, optimizer, scheduler, n_offset=1):
    total_batches = len(train_dataloader)
    train_size, train_batches = 0, 0
    train_loss, train_correct = 0, 0
    val_size, val_batches = 0, 0
    val_loss, val_correct = 0, 0
    
    with tqdm(desc=f'Epoch {epoch+n_offset}', total=total_batches) as bar:
        
        # Training
        for batch, (src, y) in enumerate(train_dataloader):
            src, y = src.to(device), y.to(device)
            
                
            
            # Compute prediction and loss
            pred = model(src)
            loss = loss_fn(pred, y)
            
            # Compute metrics
            train_loss += loss.item()
            train_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            train_size += len(y)
            train_batches += 1

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
                
            scheduler.step(epoch + batch / total_batches)
            
            # Update progress bar
            bar.update()
            bar.set_postfix(accuracy = train_correct / train_size, loss = train_loss / train_batches,
                           lr=scheduler.get_last_lr())
            
        bar.set_postfix(accuracy = train_correct / train_size, loss = train_loss / train_batches)
        
        #scheduler.step()
           
        # Validation
        with torch.no_grad():

            for batch, (src, y) in enumerate(val_dataloader):
                src, y = src.to(device), y.to(device)
                
                
                # Compute prediction and loss
                pred = model(src)
                val_loss += loss_fn(pred, y).item()
                
                # Compute metrics
                val_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
                val_size += len(y)
                val_batches += 1

                # Update progress bar
                bar.set_postfix(
                    accuracy = train_correct / train_size, loss = train_loss / train_batches,
                    val_accuracy = val_correct / val_size, val_loss = val_loss / val_batches
                )
                
    if scheduler.T_0 - scheduler.T_cur < 0.1:
        print()
        
    train_losses.append(train_loss / train_batches)
    val_losses.append(val_loss / val_batches)

    return train_correct / train_size, train_loss / train_batches, val_correct / val_size, val_loss / val_batches





#model.load_state_dict(saved_state)
# Iterate through epochs for training


best_loss = float('inf')
saved_state = custon.state_dict()

epochs = 300
for epoch in range(epochs):
    acc, loss, v_acc, v_loss = train_val_loop(epoch, train_loader, val_loader, custon, loss_fn, optimizer, scheduler)
    if v_loss<best_loss:
        best_loss = v_loss
        saved_state = custon.state_dict()

Epoch 8: 100%|██████████| 333/333 [00:05<00:00, 62.02it/s, accuracy=0.535, loss=2.13, val_accuracy=0.00847, val_loss=13.4]


Epoch 16: 100%|██████████| 333/333 [00:05<00:00, 62.18it/s, accuracy=0.553, loss=2.08, val_accuracy=0.00868, val_loss=13.5]


Epoch 24: 100%|██████████| 333/333 [00:05<00:00, 61.10it/s, accuracy=0.565, loss=2.05, val_accuracy=0.00804, val_loss=13.4]


Epoch 32: 100%|██████████| 333/333 [00:05<00:00, 65.42it/s, accuracy=0.572, loss=2.03, val_accuracy=0.00847, val_loss=13.4]


Epoch 40: 100%|██████████| 333/333 [00:05<00:00, 61.81it/s, accuracy=0.578, loss=2.02, val_accuracy=0.00836, val_loss=13.3]


...

Epoch 88: 100%|██████████| 333/333 [00:05<00:00, 64.26it/s, accuracy=0.59, loss=1.99, val_accuracy=0.0072, val_loss=12.9] 


Epoch 96: 100%|██████████| 333/333 [00:05<00:00, 63.44it/s, accuracy=0.591, loss=1.99, val_accuracy=0.00783, val_loss=12.8]


Epoch 104: 100%|██████████| 333/333 [00:05<00:00, 65.51it/s, accuracy=0.592, loss=1.98, val_accuracy=0.0072, val_loss=12.8] 


Epoch 105:  67%|██████▋   | 224/333 [00:03<00:01, 68.50it/s, accuracy=0.479, loss=2.3, lr=[0.0004922644854837775]] 


KeyboardInterrupt: 

In [27]:
import torch.optim as optim
import torch.nn.functional as F
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 650
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, 250))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    

    # Training of the model.
    for epoch in range(100):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(val_loader):
                
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / (len(val_loader.dataset))

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-04-28 01:37:10,613] A new study created in memory with name: no-name-92298d23-32bb-41f4-9e33-5c19eee3a697
[I 2023-04-28 01:38:47,264] Trial 0 finished with value: 0.0060330228619813716 and parameters: {'n_layers': 1, 'n_units_l0': 70, 'dropout_l0': 0.2035279888018733, 'optimizer': 'Adam', 'lr': 0.0068110790235364205}. Best is trial 0 with value: 0.0060330228619813716.
[I 2023-04-28 01:40:19,031] Trial 1 finished with value: 0.0034928027095681626 and parameters: {'n_layers': 2, 'n_units_l0': 109, 'dropout_l0': 0.32153480665931067, 'n_units_l1': 7, 'dropout_l1': 0.39578128187000106, 'optimizer': 'SGD', 'lr': 0.009221309986333685}. Best is trial 0 with value: 0.0060330228619813716.
[I 2023-04-28 01:41:58,367] Trial 2 finished with value: 0.004022015241320914 and parameters: {'n_layers': 2, 'n_units_l0': 128, 'dropout_l0': 0.3414935840406287, 'n_units_l1': 77, 'dropout_l1': 0.3255424802484055, 'optimizer': 'RMSprop', 'lr': 0.06580264442519955}. Best is trial 0 with value: 0.0060330

Study statistics: 
  Number of finished trials:  11
  Number of pruned trials:  4
  Number of complete trials:  7
Best trial:
  Value:  0.0073031329381879765
  Params: 
    n_layers: 1
    n_units_l0: 64
    dropout_l0: 0.20573955607494795
    optimizer: Adam
    lr: 0.0025661782346817707
